# TemplateBank

A TemplateBank is a thin wrapper on an [obsplus EventBank](https://niosh-mining.github.io/obsplus/notebooks/interfaces/eventbank.html).  TemplateBanks handle database tasks for storing, selecting and reading in catalogues of templates from/on disk. The idea is to have a suite of templates for all possible events of interest; when something interesting happens this database can be queried to provide the relevant templates for that trigger. Because `TemplateBank`s subclass `EventBank`, all the look-up methods for `EventBank` are available.

Lets have a look at how we might generate a `TemplateBank` from scratch for a day of the New Zealand national catalogue.

In [1]:
from rt_eqcorrscan.database import TemplateBank
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

bank = TemplateBank(base_path="./template_bank")
client = Client("GEONET")
catalog = client.get_events(
    starttime=UTCDateTime(2019, 6, 21),
    endtime=UTCDateTime(2019, 6, 23),
    latitude=-38.8, longitude=175.8, maxradius=0.2)
print(catalog)

7 Event(s) in Catalog:
2019-06-21T03:46:25.054642Z | -38.824, +175.865 | 2.178845358 M  | manual
2019-06-21T03:52:00.142063Z | -38.826, +175.855 | 1.230595585 M  | manual
2019-06-21T08:28:16.143864Z | -38.827, +175.866 | 1.364494767 M  | manual
2019-06-21T08:29:23.083982Z | -38.827, +175.878 | 2.066853896 M  | manual
2019-06-21T09:07:54.730731Z | -38.819, +175.904 | 1.420446524 M  | manual
2019-06-21T15:34:49.878441Z | -38.764, +175.615 | 1.530488164 M  | manual
2019-06-22T09:35:27.682975Z | -38.819, +175.866 | 1.880393466 M  | manual


First we will put the events into the database.

In [2]:
bank.put_events(catalog)
bank.read_index()

,time,latitude,longitude,depth,magnitude,event_description,associated_phase_count,azimuthal_gap,horizontal_uncertainty,local_magnitude,...,standard_error,used_phase_count,station_count,vertical_uncertainty,updated,author,agency_id,creation_time,version,path
event_id,,,,,,,,,,,,,,,,,,,,,
smi:nz.org.geonet/2019p463190,1.561108e+09,-38.819088,175.904159,5000.000000,1.420447,Taupo,32,76.976382,4347.700016,1.854579,...,0.206139,18,28,0.000000,1.563335e+09,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),1.561108e+09,None,/2019/06/21/2019-06-21T09-07-54_63190.xml
smi:nz.org.geonet/2019p463916,1.561131e+09,-38.764454,175.615082,113987.533600,1.530488,Taupo,21,265.354118,11605.852290,1.791871,...,0.406860,21,14,13899.074190,1.563335e+09,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),1.561131e+09,None,/2019/06/21/2019-06-21T15-34-49_63916.xml
smi:nz.org.geonet/2019p463116,1.561106e+09,-38.827057,175.866028,7828.764915,1.364495,Taupo,26,91.194962,5572.845242,1.738285,...,0.255632,16,19,8385.635050,1.563335e+09,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),1.561106e+09,None,/2019/06/21/2019-06-21T08-28-16_63116.xml
smi:nz.org.geonet/2019p462596,1.561089e+09,-38.826271,175.855438,5000.000000,1.230596,Taupo,21,163.628212,6603.341442,1.494102,...,0.229197,15,15,0.000000,1.563335e+09,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),1.561089e+09,None,/2019/06/21/2019-06-21T03-52-00_62596.xml
smi:nz.org.geonet/2019p462586,1.561089e+09,-38.823746,175.864609,5000.000000,2.178845,Taupo,46,67.671886,3811.082605,2.563985,...,0.279893,25,39,0.000000,1.563335e+09,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),1.561089e+09,None,/2019/06/21/2019-06-21T03-46-25_62586.xml
smi:nz.org.geonet/2019p463118,1.561106e+09,-38.827339,175.877670,5000.000000,2.066854,Taupo,61,40.233490,3076.595177,2.322847,...,0.350769,37,53,0.000000,1.563335e+09,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),1.561106e+09,None,/2019/06/21/2019-06-21T08-29-23_63118.xml
smi:nz.org.geonet/2019p465944,1.561196e+09,-38.818531,175.866409,13595.767970,1.880393,Taupo,39,56.649483,5168.447127,2.168987,...,0.478102,26,34,7484.634962,1.563335e+09,scevent@kseqp02.geonet.org.nz,WEL(GNS_Primary),1.561196e+09,None,/2019/06/22/2019-06-22T09-35-27_65944.xml


Now we want to populate the database with templates: at the moment the database only contains event
files. Note that this might take a while - it will download the required data and process it.  
The same arguments used by EQcorrscan's [Tribe.construct](https://eqcorrscan.readthedocs.io/en/latest/submodules/core.match_filter.Tribe.html#eqcorrscan.core.match_filter.Tribe.construct) 
are supported by `Template_bank.make_templates`.

In [3]:
bank.make_templates(
    catalog=catalog, client=client, lowcut=2., highcut=15.,
    samp_rate=50., filt_order=4, prepick=0.5, length=3, swin="all")

/home/calumch/miniconda3/envs/conda_36/lib/python3.6/site-packages/obspy/signal/detrend.py:31: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(data.dtype, float):
Request would result in too much data. Denied by the datacenter. Split the request in smaller parts
Detailed response of server:


/home/calumch/miniconda3/envs/conda_36/lib/python3.6/site-packages/obspy/io/mseed/core.py:772: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


Tribe of 7 templates

This both returns the `Tribe` of templates that are created, and writes them to disk for later retrieval.

In [4]:
tribe = bank.get_templates()
print(tribe)

/home/calumch/miniconda3/envs/conda_36/lib/python3.6/site-packages/obspy/core/event/base.py:688: UserWarning: no object found with resource id smi:nz.org.geonet/20190621.034627.60-AIC-NZ.RATZ.10.HHZ, returning None
  return self._get_similar_referred_object()
/home/calumch/miniconda3/envs/conda_36/lib/python3.6/site-packages/obspy/core/event/base.py:688: UserWarning: no object found with resource id smi:nz.org.geonet/20190621.034628.68-AIC-NZ.RITZ.10.EHZ, returning None
  return self._get_similar_referred_object()
/home/calumch/miniconda3/envs/conda_36/lib/python3.6/site-packages/obspy/core/event/base.py:688: UserWarning: no object found with resource id smi:nz.org.geonet/20190621.034628.70-AIC-NZ.WATZ.10.EHZ, returning None
  return self._get_similar_referred_object()
/home/calumch/miniconda3/envs/conda_36/lib/python3.6/site-packages/obspy/core/event/base.py:688: UserWarning: no object found with resource id smi:nz.org.geonet/Pick/20190621034738.581329.94392, returning None
  return s

Tribe of 7 templates


`get_templates` supports filtering using event queries:

In [5]:
bank.get_templates(starttime=UTCDateTime(2019, 6, 21, 9))

Tribe of 3 templates

## Usage in RT-EQcorrscan

A `TemplateBank` is required to use the `CatalogListener` and `Reactor` classes. Both these classes will *listen* to a remote client service for new events and add those events to the `TemplateBank`. The `Reactor` class will also react to any events that meet user-defined thresholds, collect the appropriate temlates using the `TemplateBank`, and run the real-time matched-filter process.

## Usage outside of RT-EQcorrscan

Because `TemplateBanks` return `Tribe`s, `TemplateBanks` represent an efficient means of storing `Tribes` on disk.  `Tribe`s can be used offline for matched-filter detection.